In [6]:
import os
import platform
import sys

def get_project_root() -> str:
    system = platform.system()
    return r"D:\Users\tcshore\Documents\Projects\Tangrams\Data" if system == "Windows" else "/home/tshore/Projects/tangrams-restricted/Data"

infile_path = os.path.join(get_project_root(), "wordprobseqs-inflected-small.tsv")
print("Will read file \"{}\".".format(infile_path), file=sys.stderr)

Will read file "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv".


In [7]:
import csv
import pandas as pd

RESULTS_FILE_CSV_DIALECT = csv.excel_tab
# NOTE: "category" dtype doesn't work with pandas-0.21.0 but does with pandas-0.21.1
#__RESULTS_FILE_DTYPES = {"DYAD": "category", "WORD": "category", "IS_TARGET": bool, "IS_OOV": bool,
#				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}
__RESULTS_FILE_DTYPES = {"DYAD": "category", "IS_TARGET": bool, "IS_OOV": bool,
				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}

def read_results_file(inpath: str, encoding: str) -> pd.DataFrame:
	print("Reading \"{}\" using encoding \"{}\".".format(inpath, encoding), file=sys.stderr)
	result = pd.read_csv(inpath, dialect=RESULTS_FILE_CSV_DIALECT, sep=RESULTS_FILE_CSV_DIALECT.delimiter,
						 float_precision="round_trip",
						 encoding=encoding, memory_map=True, dtype=__RESULTS_FILE_DTYPES)
	return result

cv_results = read_results_file(infile_path, "windows-1252")
print("Read {} cross-validation results for {} dyad(s).".format(cv_results.shape[0], cv_results["DYAD"].nunique()),
      file=sys.stderr)

Reading "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv" using encoding "windows-1252".
Read 144800 cross-validation results for 2 dyad(s).


In [8]:
def find_target_ref_rows(df: pd.DataFrame) -> pd.DataFrame:
	result = df.loc[df["IS_TARGET"] == True]
	result_row_count = result.shape[0]
	complement_row_count = df.loc[~df.index.isin(result.index)].shape[0]
	assert result_row_count + complement_row_count == df.shape[0]
	print("Found {} nontarget rows and {} target rows. Ratio: {}".format(complement_row_count, result_row_count,
																		 complement_row_count / float(
																			 result_row_count)), file=sys.stderr)
	return result

cv_results = find_target_ref_rows(cv_results)

Found 137560 nontarget rows and 7240 target rows. Ratio: 19.0


In [9]:
OOV_LABEL = "__OOV__"
    
print("Replacing OOV words with label \"{}\".".format(OOV_LABEL), file=sys.stderr)
cv_results.loc[cv_results["IS_OOV"] == True, "WORD"] = OOV_LABEL

Replacing OOV words with label "__OOV__".


In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#import numpy as np

# Create vocab before splitting training and testing DFs so that the word feature set is stable
#vocab_words = tuple(sorted(cv_results["WORD"].unique()))
print("Fitting one-hot encoder for vocabulary of size {}.".format(len(cv_results["WORD"].unique())), file=sys.stderr)

# https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
# integer encode
label_encoder = LabelEncoder()
vocab_labels = label_encoder.fit_transform(cv_results["WORD"])
cv_results["WORD_LABEL"] = vocab_labels
#print(vocab_labels)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
vocab_labels = vocab_labels.reshape(len(vocab_labels), 1)
onehot_encoder.fit(vocab_labels)
#assert onehot_encoder.n_values_ == len(vocab_words)
#vocab_onehot_encoded = onehot_encoder.fit_transform(vocab_labels)
#print(vocab_onehot_encoded)
# invert first example
#inverted = label_encoder.inverse_transform([np.argmax(vocab_onehot_encoded[0, :])])
#print(inverted)

Fitting one-hot encoder for vocabulary of size 222.


OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [11]:
from typing import Tuple

import numpy as np

def split_training_testing(df: pd.DataFrame, test_set_size : int) -> Tuple[pd.DataFrame, pd.DataFrame]:
	dyad_ids = df["DYAD"].unique()
	training_set_size = len(dyad_ids) - test_set_size
	if training_set_size < 1:
		raise ValueError("Desired test set size is {} but only {} dyads found.".format(test_set_size, len(dyad_ids)))
	else:
		training_set_dyads = frozenset(np.random.choice(dyad_ids, training_set_size))
		print("Training set dyads: {}".format(sorted(training_set_dyads)), file=sys.stderr)
		training_set_idxs = df["DYAD"].isin(training_set_dyads)
		training_set = df.loc[training_set_idxs]
		test_set = df.loc[~training_set_idxs]
		test_set_dyads = frozenset(test_set["DYAD"].unique())
		print("Test set dyads: {}".format(sorted(test_set_dyads)), file=sys.stderr)

		assert not frozenset(training_set["DYAD"].unique()).intersection(frozenset(test_set_dyads))
		return training_set, test_set
    
training_df, test_df = split_training_testing(cv_results, 1)
training_df = training_df.copy(deep=False)
test_df = test_df.copy(deep=False)

Training set dyads: ['15']
Test set dyads: ['1']


In [12]:
from typing import Iterator, List, Mapping

class SequenceMatrixFactory(object):

	def __init__(self, label_encoder, onehot_encoder):
		self.label_encoder = label_encoder
		self.onehot_encoder = onehot_encoder
    
	@property
	def feature_count(self) -> int:
		word_features = self.onehot_encoder.n_values_[0]
		return word_features + 2

	def __create_datapoint_feature_array(self, row: pd.Series) -> List[float]:
		#word_features = [0.0] * len(self.__vocab_idxs)
		# The features representing each individual vocabulary word are at the beginning of the feature vector
		#word_features[self.__vocab_idxs[row["WORD"]]] = 1.0
		#word_label = self.label_encoder.transform(row["WORD"])
		word_label = row["WORD_LABEL"]
		#print("Word label: {}".format(word_label), file=sys.stderr)
        # "OneHotEncoder.transform(..)" returns a matrix even if only a single value is passed to it, so get just the first (and only) row
		word_features = self.onehot_encoder.transform(word_label)[0]
		#print("Word features: {}".format(word_features), file=sys.stderr)
		# The word label for the one-hot encoding is that with the same index as the column that has a "1" value, i.e. the highest value in the vector of one-hot encoding values
		#inverse_label = np.argmax(word_features)
		#assert inverse_label == word_label
		#inverse_word = self.label_encoder.inverse_transform([inverse_label])
		#print("Inverse word label: {}".format(inverse_label), file=sys.stderr)
		is_instructor = 1.0 if row["IS_INSTRUCTOR"] else 0.0
		#is_oov = 1.0 if row["IS_OOV"] else 0.0
		#is_target = 1.0 if row["IS_TARGET"] else 0.0
		score = row["PROBABILITY"]
		other_features = np.array((is_instructor, score))
		#result = word_features + other_features
		result = np.concatenate((word_features, other_features))
		#print("Created a vector of {} features.".format(len(result)), file=sys.stderr)
		return result

	def __call__(self, df : pd.DataFrame) -> np.array:
		# https://stackoverflow.com/a/47815400/1391325
		df.sort_values("TOKEN_SEQ_ORDINALITY", inplace=True)
		sequence_groups = df.groupby(("CROSS_VALIDATION_ITER", "DYAD", "SPLIT_SEQ_NO", "UTT_START_TIME", "UTT_END_TIME", "ENTITY"), as_index=False)
		return np.array(tuple(tuple(self.__create_feature_vectors(seq)) for _, seq in sequence_groups))
        
	def __create_feature_vectors(self, df : pd.DataFrame) -> Iterator[List[float]]:
		# noinspection PyProtectedMember
		return (self.__create_datapoint_feature_array(row._asdict()) for row in df.itertuples(index=False))

print("Splitting token sequences.", file=sys.stderr)
seq_matrix_factory = SequenceMatrixFactory(label_encoder, onehot_encoder)
training_matrix = seq_matrix_factory(training_df)
print("Created a training data matrix of shape {}.".format(training_matrix.shape), file=sys.stderr)
test_matrix = seq_matrix_factory(test_df)
print("Created a test data matrix of shape {}.".format(test_matrix.shape), file=sys.stderr)

Splitting token sequences.
Created a training data matrix of shape (723, 4, 225).
Created a test data matrix of shape (1087, 4, 225).


In [13]:
training_x = training_matrix[:,:,:-1]
#print(training_x.shape)
assert len(training_x.shape) == 3
training_y = training_matrix[:,:,-1]
#print(training_y.shape)
assert len(training_y.shape) == 2

from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

model = Sequential()
#word_embeddings = Embedding(len(vocab), embedding_vector_length, input_length=max_review_length)
#model.add(word_embeddings)
# model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# input shape is a pair of (timesteps, features) <https://stackoverflow.com/a/44583784/1391325>
input_shape = training_x.shape[1:]
print("Input shape: {}".format(input_shape), file=sys.stderr)
units = training_y.shape[1]
print("Units: {}".format(units), file=sys.stderr)
lstm = LSTM(input_shape=input_shape, units=units)
#lstm = LSTM(batch_input_shape = training_x.shape, stateful = True, units=len(training_y.shape))
model.add(lstm)
model.add(Dense(units, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

(723, 4, 224)
(723, 4)


Using TensorFlow backend.
Input shape: (4, 224)
Units: 4


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4)                 3664      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
Total params: 3,684
Trainable params: 3,684
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# train LSTM
#model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
training_history = model.fit(x=training_x, y=training_y, epochs=1, verbose=1)

# train LSTM <https://machinelearningmastery.com/memory-in-a-long-short-term-memory-network/>
#epochs = 5
#for i in range(epochs):
#    x = training_x[i]
#    x_oldshape = x.shape
#    x_newshape = (-1, x_oldshape[0], x_oldshape[1])
#    x = np.reshape(x, x_newshape)
#    #print("x.shape = {}".format(x.shape), file=sys.stderr)
#    y = training_y[i]
#    y_oldshape = y.shape
#   y_newshape = (-1, y_oldshape[0])
#    y = np.reshape(y, y_newshape)
#    #print("y.shape = {}".format(y.shape), file=sys.stderr)
#    model.fit(x, y, epochs=1, batch_size=1, verbose=1, shuffle=False)
#    model.reset_states()
    
    

Epoch 1/1
723/723 [==============================] - 1s 1ms/step - loss: 0.6885 - acc: 0.2033


In [15]:
# test LSTM

def onehot_encodings(features : np.array, onehot_encoder) -> np.array:
    """
    :param features: A 2D array of feature vectors, each of which starts with one-hot encoding features. Any other features follow those.
    :param onehot_encoder: The instance used to encode the original values.
    :return: A 2D numpy array consisting only of one-hot encoding features.
    """
    feature_count = onehot_encoder.n_values_[0]
    return features[:,:feature_count + 1]


test_x = test_matrix[:,:,:-1]
print(test_x.shape)
assert len(test_x.shape) == 3
test_y = test_matrix[:,:,-1]
print(test_y.shape)
assert len(test_y.shape) == 2

seq_predicted_values = model.predict(test_x, verbose=0)
model.reset_states()
#print("result.shape = {}".format(seq_predicted_values.shape), file=sys.stderr)
for i, tested_seq in enumerate(test_x):
    word_features = onehot_encodings(tested_seq, onehot_encoder)
    #print("word features: {}".format(word_features))
    #print("Tested sequence: {}".format(tested_seq))
    #print("Tested sequence shape: {}".format(tested_seq.shape))
    seq_labels = word_features.argmax(axis=1)
    #print("Inverse labels: {}".format(inverse_labels))
    seq_words = label_encoder.inverse_transform(seq_labels)
    print("Inverse word labels: {}".format(seq_words))
    predicted_values = seq_predicted_values[i]
    assert tested_seq.shape[:-1] == predicted_values.shape
    #assert tested_seq.shape[:-1] == actual_values.shape
    #rint("Predicted values: {}".format(predicted_values))
    actual_values = test_y[i]
    assert predicted_values.shape == actual_values.shape
    
    differences = predicted_values - actual_values
    print("[Predicted_val - Actual_val]: {}".format(differences))
    #print("Actual values: {}".format(actual_values))
	#print('X=%.1f y=%.1f, yhat=%.1f' % (seq1[i], seq1[i+1], result[i]))

(1087, 4, 224)
(1087, 4)
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'ah']
[Predicted_val - Actual_val]: [ 0.48070422  0.4449276   0.45813471 -0.12466304]
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'waiting']
[Predicted_val - Actual_val]: [ 0.47571787  0.45568606  0.46887767  0.48110041]
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'so']
[Predicted_val - Actual_val]: [ 0.47425687  0.45473468  0.47438207  0.11670886]
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'it']
[Predicted_val - Actual_val]: [ 0.47789916  0.44196486  0.46810937 -0.0210595 ]
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'yes']
[Predicted_val - Actual_val]: [ 0.48399571  0.45714062  0.47607592 -0.0536976 ]
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'the']
[Predicted_val - Actual_val]: [ 0.47412872  0.44649675  0.4653466  -0.02332622]
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'per

[Predicted_val - Actual_val]: [ 0.47587726  0.46461412 -0.16435501 -0.14351219]
Inverse word labels: ['__PADDING__' '__PADDING__' 'I' 'I']
[Predicted_val - Actual_val]: [ 0.4871603   0.46057919 -0.06577103 -0.06763433]
Inverse word labels: ['__PADDING__' '__PADDING__' 'so' 'it']
[Predicted_val - Actual_val]: [ 0.47265333  0.44908887  0.0237953   0.03035435]
Inverse word labels: ['__PADDING__' '__PADDING__' 'it' "'s"]
[Predicted_val - Actual_val]: [ 0.46908775  0.44612947  0.01816851  0.03925811]
Inverse word labels: ['__PADDING__' '__PADDING__' 'yeah' 'exactly']
[Predicted_val - Actual_val]: [ 0.48092481  0.44063759 -0.01107876  0.16133138]
Inverse word labels: ['__PADDING__' '__PADDING__' 'and' 'yeah']
[Predicted_val - Actual_val]: [ 0.46952894  0.44230416  0.13802665 -0.02914176]
Inverse word labels: ['__PADDING__' '__PADDING__' 'just' 'where']
[Predicted_val - Actual_val]: [ 0.48181173  0.45476973  0.19515213  0.33098311]
Inverse word labels: ['__PADDING__' '__PADDING__' 'just' 'abo

Inverse word labels: ['yeah' 'it' "'s" 'the']
[Predicted_val - Actual_val]: [-0.06577214 -0.04957728 -0.0270376  -0.00365107]
Inverse word labels: ['yeah' 'this' 'time' 'we']
[Predicted_val - Actual_val]: [-0.00657551 -0.0104671   0.46399277  0.37064882]
Inverse word labels: ['they' 'are' 'light' 'blue']
[Predicted_val - Actual_val]: [ 0.47598088 -0.32470924 -0.15104699 -0.20342656]
Inverse word labels: ['now' 'it' "'s" 'the']
[Predicted_val - Actual_val]: [-0.01709892 -0.06691081 -0.04020211 -0.02128659]
Inverse word labels: ['so' 'this' 'is' 'a']
[Predicted_val - Actual_val]: [-0.07684062 -0.06586355 -0.09541181 -0.18053188]
Inverse word labels: ['it' "'s" 'small' 'and']
[Predicted_val - Actual_val]: [-0.05661255 -0.05590523  0.16118048 -0.14718234]
Inverse word labels: ['it' "'s" 'a' 'W']
[Predicted_val - Actual_val]: [-0.0574978  -0.0634715  -0.20411636 -0.47722233]
Inverse word labels: ['I' 'would' 'say' 'a']
[Predicted_val - Actual_val]: [-0.06940502  0.45093915  0.11607006 -0.18

Inverse word labels: ['is' 'what' 'some' '__OOV__']
[Predicted_val - Actual_val]: [-0.03078465  0.16398966  0.4755308  -0.03258994]
Inverse word labels: ['symbol' 'the' 'green' 'V']
[Predicted_val - Actual_val]: [ 0.09084617  0.01860376 -0.01886161 -0.44858295]
Inverse word labels: ['the' 'center' 'there' 'is']
[Predicted_val - Actual_val]: [-0.00944904 -0.32357605 -0.01843214 -0.04441629]
Inverse word labels: ['a' 'deer' 'or' 'a']
[Predicted_val - Actual_val]: [ 0.05335444  0.20870474 -0.00973399  0.05500114]
Inverse word labels: ["'s" 'the' 'the' 'one']
[Predicted_val - Actual_val]: [-0.02067208 -0.04234943 -0.03106597  0.10881925]
Inverse word labels: ['of' 'the' 'man' 'in']
[Predicted_val - Actual_val]: [ 0.31660747 -0.03131432  0.36610197  0.12056152]
Inverse word labels: ['big' 'it' "'s" 'quite']
[Predicted_val - Actual_val]: [-0.11495934 -0.00707775 -0.00876471  0.48105323]
Inverse word labels: ["'s" 'a' 'yellow' 'thing']
[Predicted_val - Actual_val]: [-0.0155037  -0.17252875 -0